In [1]:
import sympy as sp
from sympy import *
from sympy.abc import *
from sympy.parsing.sympy_parser import parse_expr
from itertools import combinations

In [2]:
CNF_FILE_NAME = f'graph_{1}.cnf'

In [3]:
def readIntoAM(filename: str):
    am = []
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            line = list(map(int, line[:-1]))
            am.append(list(line))
    for i in range(len(am)):
        for j in range(i, len(am[i])):
            am[i][j] = am[j][i]
    return am
    

In [4]:
adjaceny_matrix = am = readIntoAM(f'./am/testcase_{1}.cov')

In [5]:
am

[[0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 1],
 [0, 1, 1, 0, 1, 1, 0],
 [1, 0, 0, 1, 0, 0, 1],
 [1, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 1, 1, 0]]

In [6]:
# adjaceny_matrix = am = [
#             [0, 1, 1, 1, 1],
#             [1, 0, 0, 0, 1],
#             [1, 0, 0, 1, 1],
#             [1, 0, 1, 0, 1],
#             [1, 1, 1, 1, 0]
# ]

In [7]:
max_num = len(am)

In [8]:
def define_variables(max_no: int):
    return symbols(f'x:{max_no}')

In [9]:
syms: sp.Symbol = define_variables(max_num)
for i in range(max_num):
    exec(f'x{i} = syms[{i}]')

In [10]:
clauses = []

In [11]:
# add edges to our clauses
for i in range(max_num):
    for j in range(i+1):
        if am[i][j] == 1:
            clauses.append(f'x{i}|x{j}')

In [12]:
clauses

['x3|x1',
 'x3|x2',
 'x4|x0',
 'x4|x3',
 'x5|x0',
 'x5|x1',
 'x5|x3',
 'x6|x2',
 'x6|x4',
 'x6|x5']

In [13]:
# encode addition from 1 to k
# m = len(clauses)
k = 4
overall_addition = []
se = set()
for symbol in syms:
    se.add(str(symbol))

for i in range(1, k+1):
    x = list(combinations(se, i))
    clause_string = ''
    for part_clause in x:
        clause_string += '&'.join(part_clause) 
        negated = se - set(part_clause)
        if len(negated) != 0:
            clause_string += '&~'+'&~'.join(negated)
        clause_string += '|'
    clause_string = clause_string[:-1]
    overall_addition.append(clause_string)

exec('overall_addition_course = ' + '|'.join(overall_addition))
sp.to_cnf(overall_addition_course, True)

(x0 | x1 | x2 | x3 | x4 | x5 | x6) & (~x0 | ~x1 | ~x2 | ~x3 | ~x4) & (~x0 | ~x1 | ~x2 | ~x3 | ~x5) & (~x0 | ~x1 | ~x2 | ~x3 | ~x6) & (~x0 | ~x1 | ~x2 | ~x4 | ~x5) & (~x0 | ~x1 | ~x2 | ~x4 | ~x6) & (~x0 | ~x1 | ~x2 | ~x5 | ~x6) & (~x0 | ~x1 | ~x3 | ~x4 | ~x5) & (~x0 | ~x1 | ~x3 | ~x4 | ~x6) & (~x0 | ~x1 | ~x3 | ~x5 | ~x6) & (~x0 | ~x1 | ~x4 | ~x5 | ~x6) & (~x0 | ~x2 | ~x3 | ~x4 | ~x5) & (~x0 | ~x2 | ~x3 | ~x4 | ~x6) & (~x0 | ~x2 | ~x3 | ~x5 | ~x6) & (~x0 | ~x2 | ~x4 | ~x5 | ~x6) & (~x0 | ~x3 | ~x4 | ~x5 | ~x6) & (~x1 | ~x2 | ~x3 | ~x4 | ~x5) & (~x1 | ~x2 | ~x3 | ~x4 | ~x6) & (~x1 | ~x2 | ~x3 | ~x5 | ~x6) & (~x1 | ~x2 | ~x4 | ~x5 | ~x6) & (~x1 | ~x3 | ~x4 | ~x5 | ~x6) & (~x2 | ~x3 | ~x4 | ~x5 | ~x6)

In [14]:
# se

In [15]:
overall_addition_course_cnf = sp.to_cnf(overall_addition_course, True)

In [16]:
# overall_addition_course_cnf

In [17]:
clauses.append(str(overall_addition_course_cnf))

In [18]:
# clauses

In [19]:
exec('a='+'('+')&('.join(clauses) + ')')

In [20]:
# sp.to_cnf(a, True)

In [21]:
simplify(a)

(x0 | x4) & (x0 | x5) & (x1 | x3) & (x1 | x5) & (x2 | x3) & (x2 | x6) & (x3 | x5) & (x4 | x6) & (~x0 | ~x4) & (~x2 | ~x6) & (x0 | ~x1 | ~x3) & (x2 | ~x1 | ~x5) & (~x1 | ~x3 | ~x5)

In [22]:
satisfiable(a)

{x1: True, x2: True, x4: True, x5: True, x6: False, x3: False, x0: False}

In [23]:
# a

In [24]:
def write_cnf(filename, cnf_ans, set_of_vars):
    assert cnf_ans.func == sp.And
    l = []
    for row in am:
        l.append(f'c {row}')
    l.append(f'p cnf {len(set_of_vars)} {len(cnf_ans.args)}')
    for arg in cnf_ans.args:
        s = ''
        if arg.func == sp.Symbol:
            s += str(set_of_vars[arg]) + ' '
        else:
            assert arg.func == sp.Or
            for sub_arg in arg.args:
                if len(sub_arg.args) == 1:
                    s += str(-1*(set_of_vars[sub_arg.args[0]])) + ' '
                elif len(sub_arg.args) == 0:
                    s += str(set_of_vars[sub_arg]) + ' '
        s += '0'
        l.append(s)
    with open(filename,'w') as f:
        f.write('\n'.join(l))

In [25]:
def set_wrapper(cnf_ans):
    set_of_vars = {}
    counter = 1
    def recursive_add_to_set(a):
        nonlocal counter
        if a.func == sp.Symbol:
            if a not in set_of_vars:
                set_of_vars[a] = counter
                counter += 1
        elif len(a.args)==1:
            b = a.args[0]
            if b not in set_of_vars:
                set_of_vars[b] = counter
                counter += 1
        else:
            for arg in a.args:
                recursive_add_to_set(arg)
    recursive_add_to_set(cnf_ans)
    return set_of_vars

In [26]:
write_cnf(CNF_FILE_NAME, a, set_wrapper(a))